# Worksheet 5-1: Nonlinear Regression

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/WCC-Engineering/ENGR240/blob/main/Class%20Demos%20and%20Activities/Week%205/Worksheet%205-1_template%20nonlinear%20regression%20(final).ipynb)

## Overview

In this worksheet, we'll explore three different approaches to fit nonlinear models to data:

1. **Linear regression on transformed data**: Converting a nonlinear model to linear form
2. **Nonlinear regression using scipy.optimize.minimize**: Direct optimization of the sum of squared residuals
3. **Nonlinear regression using scipy.optimize.curve_fit**: A more convenient wrapper for nonlinear regression

We'll apply these techniques to a bacteria growth rate model and compare their results and implementation complexity.

## The Model

We'll work with a bacterial growth rate model that describes how the growth rate depends on substrate concentration:

$$k = k_{max} \frac{c^2}{c_s + c^2}$$

where:
- $k$ is the growth rate (number/day)
- $c$ is the substrate concentration (mg/L)
- $k_{max}$ is the maximum possible growth rate
- $c_s$ is the half-saturation constant

Our goal is to find the values of $k_{max}$ and $c_s$ that best fit our experimental data.

In [ ]:
# Import necessary libraries
import numpy as np
from scipy import optimize
import matplotlib.pyplot as plt

# Set plotting style
plt.style.use('seaborn-v0_8-whitegrid')

# Set random seed for reproducibility
np.random.seed(42)

## Experimental Data

Let's start with our experimental measurements of bacterial growth rate at different substrate concentrations:

In [ ]:
# Substrate concentration (mg/L)
c = np.array([0.5, 0.8, 1.5, 2.5, 4.0])

# Growth rate (number/day)
k = np.array([1.0, 2.5, 5.1, 7.3, 9.1])

# Plot the experimental data
plt.figure(figsize=(10, 6))
plt.scatter(c, k, color='black', s=50, label='Experimental data')
plt.xlabel('Substrate Concentration (mg/L)')
plt.ylabel('Growth Rate (number/day)')
plt.title('Bacterial Growth Rate vs. Substrate Concentration')
plt.legend()
plt.grid(True)
plt.show()

Let's define our model function that we'll use throughout this worksheet:

In [ ]:
def kmodel(c, kmax, cs):
    """Bacterial growth rate model
    
    Parameters:
    -----------
    c : array_like
        Substrate concentration (mg/L)
    kmax : float
        Maximum growth rate (number/day)
    cs : float
        Half-saturation constant
        
    Returns:
    --------
    k : array_like
        Growth rate (number/day)
    """
    return kmax * c**2 / (cs + c**2)

## Task 1: Linear Regression with Transformed Data

Often, we can transform a nonlinear model into a linear form that allows us to use simple linear regression. For our model:

$$k = k_{max} \frac{c^2}{c_s + c^2}$$

### 1.1 Derive the Linear Transformation

In the cell below, derive a linear transformation of this model. Hint: Start by taking the reciprocal (1/k) of both sides.

**Your derivation here:**

Taking the reciprocal of both sides:

$$\frac{1}{k} = \frac{c_s + c^2}{k_{max} \cdot c^2} = \frac{c_s}{k_{max} \cdot c^2} + \frac{1}{k_{max}}$$

This is now in the form of a linear equation:

$$\frac{1}{k} = \frac{c_s}{k_{max}} \cdot \frac{1}{c^2} + \frac{1}{k_{max}}$$

Which has the form $Y = mX + b$ where:
- $Y = \frac{1}{k}$
- $X = \frac{1}{c^2}$
- $m = \frac{c_s}{k_{max}}$
- $b = \frac{1}{k_{max}}$

### 1.2 Implement the Linear Regression

Now, implement the linear regression using the transformed variables and NumPy's `polyfit` function.

In [ ]:
# Transform the data
X = 1/c**2  # X = 1/c²
Y = 1/k     # Y = 1/k

# Use np.polyfit to perform linear regression
# Complete the line below to fit a 1st-degree polynomial (straight line) to X and Y
p = np.polyfit(X, Y, 1)

# Extract the slope (m) and intercept (b)
m, b = p

# Calculate kmax and cs from m and b
# Remember: b = 1/kmax and m = cs/kmax
kmax1 = 1/b
cs1 = m * kmax1

print(f"Linear Regression Results:")
print(f"m = {m:.4f}, b = {b:.4f}")
print(f"kmax = {kmax1:.4f}")
print(f"cs = {cs1:.4f}")

### 1.3 Visualize the Linear Fit

Let's visualize both the transformed data with the linear fit and the original data with the nonlinear model using our fitted parameters.

In [ ]:
# Create a figure with two subplots
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Plot the transformed data and linear fit
ax1.scatter(X, Y, color='black', s=50)
X_line = np.linspace(min(X), max(X), 100)
Y_line = m * X_line + b
ax1.plot(X_line, Y_line, 'r-', linewidth=2)
ax1.set_xlabel('1/c² (L²/mg²)')
ax1.set_ylabel('1/k (day/number)')
ax1.set_title('Transformed Data with Linear Fit')
ax1.grid(True)

# Plot the original data and nonlinear model
ax2.scatter(c, k, color='black', s=50)
c_line = np.linspace(0, max(c)*1.2, 100)
k_line = kmodel(c_line, kmax1, cs1)
ax2.plot(c_line, k_line, 'r-', linewidth=2)
ax2.set_xlabel('Substrate Concentration (mg/L)')
ax2.set_ylabel('Growth Rate (number/day)')
ax2.set_title('Original Data with Nonlinear Model')
ax2.grid(True)

plt.tight_layout()
plt.show()

## Task 2: Fit Quality Statistics

Now that we have our first fit, let's evaluate its quality using standard statistical measures. The two most common metrics for assessing curve fit quality are:

1. **Coefficient of Determination (R²)**: Indicates the proportion of variance in the dependent variable that is predictable from the independent variable. R² ranges from 0 to 1, where 1 indicates perfect prediction.

2. **Standard Error of the Estimate (Syx)**: Represents the average distance between the observed values and the predicted values. Lower values indicate better fit.

### 2.1 Calculate Fit Quality Metrics

For our nonlinear model, we need to calculate these metrics using the original (non-transformed) data, even though we performed the fit on transformed data.

In [ ]:
# Calculate predicted values using our model with kmax1 and cs1
k_pred1 = kmodel(c, kmax1, cs1)

# Calculate residuals (differences between observed and predicted values)
residuals = k - k_pred1

# Sum of Squared Residuals (Sr)
Sr = np.sum(residuals**2)

# Total Sum of Squares (St) - variation around the mean
St = np.sum((k - np.mean(k))**2)

# Coefficient of Determination (R²)
r_squared1 = 1 - Sr/St

# Standard Error of the Estimate (Syx)
# n-2 represents degrees of freedom (n data points minus 2 parameters)
Syx_1 = np.sqrt(Sr/(len(k)-2))

print(f"Fit Quality Metrics for Linear Regression Approach:")
print(f"Sum of Squared Residuals (Sr) = {Sr:.4f}")
print(f"Total Sum of Squares (St) = {St:.4f}")
print(f"Coefficient of Determination (R²) = {r_squared1:.4f}")
print(f"Standard Error of the Estimate (Syx) = {Syx_1:.4f}")